**Importación de librerias**

In [1]:
import pandas as pd
import numpy as np

**Lectura de datos**

In [17]:
data = pd.read_csv('Raw/data_raw.csv')
data.head(2)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,Valoraciones,Promedio_Valoraciones
0,as7362,Movie,"Drunk, Stoned, Brilliant, Dead: The Story of t...",Douglas Tirola,"Chevy Chase, Judd Apatow, Henry Beard, PJ O’Ro...",NaN,NaN,2015,R,95 min,Documentary,"From 1970s-1990s, the hippest and most outrage...",469,3.7
1,ns3234,Movie,Pranaam,Sanjiv Jaiswal,"Rajeev Khandelwal, Samiksha Singh, S.M. Zaheer...",India,"November 25, 2019",2019,TV-14,117 min,"Action & Adventure, Dramas, International Movies",Aspiring to fulfill his father’s dream and bec...,490,3.7


### **Fechas***

Segun la consigna formateamos las fechas

In [3]:
#Eliminamos los espacios en blanco que pueden haber al inicio o final de la fecha
data['date_added'] = data['date_added'].str.strip()
#Normalizo el formato de las fechas: 
data["date_added"] = pd.to_datetime(data["date_added"], format='%B %d, %Y')

### **Nulos**

Cambiamos los nulos por 'None' para evitar problemas futuros en el modelo de machine learning

In [4]:
# Eliminar los registros con valores nulos en la columna 'mi_columna'
data = data.dropna(subset=['title'])

In [5]:
#Cambiamos nulos por "None"
data = data.fillna("None")

In [6]:
#resolvemos los valores nulos de director
data.director = data.director.replace("1", "None")

### **Rating***

-En la columna rating existian datos que correspondian a duration por lo cual los cambiamos

-Segun la consigna estandarizamos todos los ratings nulos a el string 'G'

In [7]:
#Filtramos los indices de datos desplazados que corresponden a duration pero se encuentran en rating
lst = [str(i) for i in data.rating.unique()]
elementos_filtrados = [elemento for elemento in lst if "min" in elemento or "Season" in elemento]

filtrado = data.loc[data.rating.isin(elementos_filtrados)]

filtrado_i = list(filtrado.index)

#Arreglamos los datos desplazados que corresponden a duration pero se encuentran en rating
data["duration"].loc[filtrado_i] = data["rating"].loc[filtrado_i]
data["rating"] = data["rating"].drop(index = filtrado_i)

#Estandarizamos todos los valores nulos por el valor G
e_nulas = ["NR", "None", "UNRATED", "NOT_RATE", "TV-NR", "NOT RATED", "UR", np.nan]
data.rating = data.rating.replace(e_nulas, "G")


C:\Users\Ale\AppData\Local\Temp/ipykernel_16316/230770865.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["duration"].loc[filtrado_i] = data["rating"].loc[filtrado_i]


### **Duracion***

Cumpliendo la consigna separamos la columna 'duration' en dos columnas, una con los datos numericos y la otra con el tipo de duracion

In [8]:
#Creamos la columna duration_int y duration_type
data["duration_int"] = data.duration
data["duration_type"] = data.duration
data.duration_int = data.duration_int.replace('[^0-9]', '', regex=True)
data.duration_type = data.duration_type.replace('\d', '', regex=True)

#Eliminamos la columna duration original
data = data.drop("duration", axis=1)

data.head(1)

,show_id,type,title,director,cast,country,date_added,release_year,rating,listed_in,description,Valoraciones,Promedio_Valoraciones,duration_int,duration_type
0,as7362,Movie,"Drunk, Stoned, Brilliant, Dead: The Story of t...",Douglas Tirola,"Chevy Chase, Judd Apatow, Henry Beard, PJ O’Ro...",None,None,2015,R,Documentary,"From 1970s-1990s, the hippest and most outrage...",469,3.7,95,min


Convertimos los tipos de datos a los mas conveniente y aprovechamos para hacerlo en todo el dataframe

In [9]:
#Acomodamos los tipos de datos correctos para cada columna
data = data.convert_dtypes()
data["duration_int"] = data["duration_int"].replace(["None", ""], "0")
data["duration_int"] = data["duration_int"].astype(int)
data["cast"] = data["cast"].astype(object)

data.dtypes

show_id                   string
type                      string
title                     string
director                  string
cast                      object
country                   string
date_added                object
release_year               Int64
rating                    string
listed_in                 string
description               string
Valoraciones               Int64
Promedio_Valoraciones    Float64
duration_int               int32
duration_type             string
dtype: object

Dividimos los datos entre peliculas y series, tambien tomamos las peliculas que tenian "Season" y las cambiamos por la cateoria 'TV Show'

In [10]:
#eliminamos los espacios sobrantes y dividimos los datos entre peliculas y series
data['duration_type'] = data['duration_type'].str.strip()
series = data.loc[data.type == "TV Show"]
movies = data.loc[data.type == "Movie"]

#Buscamos las peliculas que tienen Season en duration_type y le cambiamos su etiqueta type
fake_movies = movies.loc[movies.duration_type.isin(["Season", "Seasons"])].index
data.type.loc[fake_movies] = data.type.loc[fake_movies].replace("Movie", "TV Show")

Aqui filtramos los datos de peliculas, reduciendolos solo a largometrajes (+60min) y creamos dos columnas categoricas binarias en base a la media de la duracion y la antiguedad

In [11]:
#Consideramos como largometrajes solo los titulos con mas de 60 min
movies = movies.loc[data.duration_int >= 60]

#Creamos una columna de clasificacion para saber si el titulo es Corto o Largo en duración
movies["duration_category"] = np.where(movies.duration_int <= movies.duration_int.mean(), "Corta", "Larga")
series["duration_category"] = np.where(series.duration_int <= series.duration_int.mean(), "Corta", "Larga")

#Creamos una columna de clasificacion para saber si el titulo tiene mas o menos de 5 años
movies["antiquity_category"] = np.where(movies.release_year <= 2018, "Vieja", "Nueva")
series["antiquity_category"] = np.where(series.release_year <= 2018, "Vieja", "Nueva")

#Unimos los datasets
data = pd.concat([movies, series])

C:\Users\Ale\AppData\Local\Temp/ipykernel_16316/4093207900.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  series["duration_category"] = np.where(series.duration_int <= series.duration_int.mean(), "Corta", "Larga")
C:\Users\Ale\AppData\Local\Temp/ipykernel_16316/4093207900.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  series["antiquity_category"] = np.where(series.release_year <= 2018, "Vieja", "Nueva")


Estandarizamos el tipo de duracion 'Season'

In [12]:
#Establecemos solo dos valores para duration_type
data.duration_type = data.duration_type.replace('Seasons', 'Season', regex=True)

### **Actores y Generos**

Transformacion de las columnas con varios elementos de string a datos de tipo lista

In [18]:
#Separamos el casting para poder manipular cada actor independientemente
data.cast = data.cast.fillna("")
data.cast = data.cast.apply(lambda x: x.split(", "))

data.listed_in = data.listed_in.fillna("")
data.listed_in = data.listed_in.apply(lambda x: x.split(", "))

data.head(2)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,Valoraciones,Promedio_Valoraciones
0,as7362,Movie,"Drunk, Stoned, Brilliant, Dead: The Story of t...",Douglas Tirola,"[Chevy Chase, Judd Apatow, Henry Beard, PJ O’R...",NaN,NaN,2015,R,95 min,[Documentary],"From 1970s-1990s, the hippest and most outrage...",469,3.7
1,ns3234,Movie,Pranaam,Sanjiv Jaiswal,"[Rajeev Khandelwal, Samiksha Singh, S.M. Zahee...",India,"November 25, 2019",2019,TV-14,117 min,"[Action & Adventure, Dramas, International Mov...",Aspiring to fulfill his father’s dream and bec...,490,3.7


Separación de datos en diferentes Dataframes para disponibilizacion de datos

In [14]:
#Hacemos una lista con todos los actores y generos unicos que hay mencionados en los datos
unique_actors = data['cast'].explode().unique()
unique_genres = data['listed_in'].explode().unique()

#Transformamos los arrays en Dataframes para luego exportarlos
actores = pd.DataFrame({'Actor': unique_actors})
generos = pd.DataFrame({'Genero': unique_genres})

In [22]:
actores.head()

,Actor
0,Chevy Chase
1,Judd Apatow
2,Henry Beard
3,PJ O’Rourke
4,Rajeev Khandelwal


In [23]:
generos.head()

,Genero
0,Documentary
1,Action & Adventure
2,Dramas
3,International Movies
4,Comedy


### **Reorden Columnas**

Reordenamos las columnas antes de exportarlas

In [15]:
#Reordenamos las columnas
data = data.reindex(columns=['show_id', 'type', 'title', 'director', 'cast', 'country', 'date_added',
       'release_year', "antiquity_category", 'rating', 'duration_int',
       'duration_type', "duration_category", 'listed_in', 'description',
       'Valoraciones', 'Promedio_Valoraciones'])

data.head(1)

,show_id,type,title,director,cast,country,date_added,release_year,antiquity_category,rating,duration_int,duration_type,duration_category,listed_in,description,Valoraciones,Promedio_Valoraciones
0,as7362,Movie,"Drunk, Stoned, Brilliant, Dead: The Story of t...",Douglas Tirola,"[Chevy Chase, Judd Apatow, Henry Beard, PJ O’R...",None,None,2015,Vieja,R,95,min,Corta,[Documentary],"From 1970s-1990s, the hippest and most outrage...",469,3.7


### **Exportacion de Datos**

Exportamos los datos en formato CSV para que puedan ser utilizados en el modelo de machine learning


In [16]:
#Exportamos los dataframes que nos interesan a CSV
data.to_csv('Datos/data.csv', index=False)
actores.to_csv('Datos/actores.csv', index=True)
generos.to_csv('Datos/generos.csv', index=True)